# Language Modeling with Keras

In [1]:
# Handles Importing all the necessary libraries
import itertools
import h5py
import numpy as np
import string
import keras
from keras.layers import Input, Embedding, merge, Flatten, Reshape, Lambda, LSTM, Dropout, Dense
from keras.layers.recurrent import SimpleRNN
import keras.backend as K
from keras.models import Model
import sklearn
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 15, 6

Using TensorFlow backend.


### Prepares the Data Set to be used

In [3]:
convos = []
# imports datasets
with open('ignored_assets/lang-train-data-simple.txt') as f:
    content = f.readlines()

# takes an array of sentences and splits it into groupings of sentences
def split_into_convo(arr, elem):
    array = arr[:]
    indiv_convos = []
    for i in range(len(arr)-1, -1, -1):
        if(array[i][0] == elem):
            indiv_convos += [array[i+1:]]
            array = array[:i]
    indiv_convos += [array]
    return indiv_convos

# you may also want to remove whitespace characters like `\n` at the end of each line,
# also strips the punctuation
sentences = [x.strip().translate(None, string.punctuation) for x in content] 

lemma = lambda x: x.strip().lower().split(' ')
sentences_lemmatized = [lemma(sentence) for sentence in sentences]
words = set(itertools.chain(*sentences_lemmatized))
# set(['boy', 'fed', 'ate', 'cat', 'kicked', 'hat'])

# dictionaries for converting words to integers and vice versa
word2idx = dict((v, i) for i, v in enumerate(words))
idx2word = list(words)

# convert the sentences a numpy array
to_idx = lambda x: [word2idx[word] for word in x]
sentences_idx = [to_idx(sentence) for sentence in sentences_lemmatized]
# Sets the maximum word length of each sentence
max_len = 9
# a list of all the indices I remove that are longer than max_len
indices_removed = []
# If the sentence is too long, good by
for i in range(len(sentences_idx)-1, -1, -1):
    if len(sentences_idx[i]) > max_len:
        indices_removed = indices_removed + [i]
        sentences_idx.pop(i)
    elif len(sentences_idx[i]) < max_len:
        sentences_idx[i] = sentences_idx[i] + [0] * (max_len - len(sentences_idx[i]));

print "Indexes Removed: {}".format(len(indices_removed))
print "Number of Sentences: {}".format(len(sentences_idx))
print "Vocabulary: {}".format(len(word2idx))
sentences_array = sentences_idx#, dtype='int32'

# splits the array into an array of grouped sentences (conversations)
conversation_array = split_into_convo(sentences_array, word2idx[''])

# Prepares the datasets as input and output
dataX = []
dataY = []
for i in range(len(conversation_array)):
    dataX += conversation_array[i][:len(conversation_array[i])-1]
    dataY += conversation_array[i][1:]
dataX = np.array(dataX).copy().astype('int32')
dataY = np.array(dataY).copy().astype('int32')

# scales the output
scaler = MinMaxScaler(feature_range=(-6, 6))
dataY = scaler.fit_transform(dataY)
# reshapes the dataY
new_dataY = dataY.reshape((1,) + dataY.shape)


Indexes Removed: 0
Number of Sentences: 121
Vocabulary: 142


/Users/dankmaster69/python-neural-nets/venv/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


## Create and Train The Model

In [10]:
n_words = len(words)
n_embed_dims = 5

# put together a model to predict
input_sentence = Input(shape=(max_len,), dtype='int32')
input_embedding = Embedding(n_words, n_embed_dims)(input_sentence)
#x = Dropout(0.3)(input_enbedding)
#rnn_layer = SimpleRNN(2*max_len)(x)
rnn_layer = SimpleRNN(2*max_len)(input_embedding)
output = Dense(max_len)(rnn_layer)

def custom_loss(y_true, y_pred):
    return K.sqrt(K.abs(y_true - y_pred))

model = Model(inputs=[input_sentence], outputs=[output])
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0), 
            loss='mae')
model.load_weights('model-simple.h5')

model.fit(dataX, dataY, epochs=2, verbose=3)
embeddings = model.layers[1].get_weights()

# print out the embedding vector associated with each word
for i in range(2):
    print('{}: {}'.format(idx2word[i], embeddings[0][i]))
    
print "Evaluation of the Model"
print model.evaluate(dataX, dataY)
print "Saving the Weights"
model.save_weights('model-simple.h5')

Epoch 1/2
Epoch 2/2
: [-3.06839633  0.64433104  2.64613485  1.48041105  1.79417467]
i’m: [ 0.04920704  0.08546274 -1.67241442 -1.75953138  0.9270339 ]
Evaluation of the Model
32/78 [===========>..................] - ETA: 0s0.864649826135
Saving the Weights


## Extra Training For The Model

In [12]:
model.fit(dataX, dataY, epochs=5000, batch_size=16, verbose=0)
print "  {}".format(model.evaluate(dataX, dataY))
print "saving the weights"
model.save_weights('model-simple.h5')

32/78 [===========>..................] - ETA: 0s  0.822150010329
saving the weights


## Exporting the Model
This is how you export the model into a json file in order to be imported later. Then you export the model's weights. Later on in other experiments, you could effectively 

In [6]:
# serialize model to JSON
model_json = model.to_json()
with open("model-simple.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model-simple.h5")

## Importing the Model
This is how you import the model from a json file and the weights so that you don't need to train it every time.

In [ ]:
# load json and create model
json_file = open('model-simple.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model-simple.h5")
print("Loaded model from disk")

## Testing the Model with Custom Input

In [150]:
# TODO - Add a check for if the word is in word2idx
# put custom words here
sent = ['i', 'love', 'you', 'too']
custom_in = [0] * max_len
for i in range(len(sent)):
    custom_in[i] = word2idx[sent[i]]
pred = model.predict(np.array(custom_in).reshape(1, max_len) )
print "Pred Values: {}".format(pred)
p = scaler.inverse_transform(pred)
print '\n\n'
for i in range(9):
    if(round(p[0][i]) != 0):
        print idx2word[ int(round(p[0][i]))]

Pred Values: [[  8.00635223e+01   7.46468735e+01   5.68069305e+01   5.83343811e+01
    3.00547719e+00   1.45626739e-02   4.47865576e-03   8.82884860e-03
   -1.67539306e-02]]





IndexError: list index out of range